# **Técnicas Avançadas de Prompt Engineering com LangChain**

**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook explora técnicas avançadas de prompt engineering aplicadas ao mercado financeiro, implementando os padrões **Reflexion** e **Self-Consistency** usando LangChain e OpenAI. O projeto demonstra como melhorar a qualidade e confiabilidade das respostas de LLMs através de:

- **Reflexion**: Técnica de autorreflexão em três etapas (geração → crítica → melhoria) que permite ao modelo revisar e aprimorar suas próprias análises
- **Self-Consistency**: Geração de múltiplas respostas com diferentes temperaturas e agregação dos resultados para identificar padrões consistentes
- **Prompt Registry**: Gerenciamento centralizado de prompts usando LangSmith para versionamento e compartilhamento
- **LCEL (LangChain Expression Language)**: Composição de chains complexas de forma declarativa e funcional

O notebook inclui casos práticos de análise de investimentos, validação de estratégias de trading e recomendações financeiras, demonstrando como essas técnicas produzem análises mais robustas, identificam riscos subestimados e geram recomendações com maior nível de confiança.

## **Instalação dos Pacotes**

In [1]:
%%capture --no-stderr
!pip install langchain langchain-openai langchain-core langchain-community
!pip install langsmith langgraph

## **Imports dos Pacotes Usados no Notebook**

Este conjunto de imports configura um ambiente para criar **agentes de IA conversacionais** usando LangChain e modelos da OpenAI. Inclui ferramentas para
- manipulação de arquivos e sistema operacional (`os`),
- acesso ao Google Drive e dados do Colab (`google.colab`),
- toda a infraestrutura do LangChain

In [2]:
import os

from google.colab import userdata

from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain.prompts import BasePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnablePassthrough

## **Configuração das Variáveis de Ambiente**

Configuração de variáveis de ambiente para OpenAI e LangSmith

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
os.environ["LANGCHAIN_PROJECT"] = "prompt-engineering-techniques"

##**Criação do principal LLM para responder as pergutnas**

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY")
)

##**Reflexion**

**Reflexion** é uma técnica de prompt engineering onde o modelo de linguagem é instruído a **analisar sua própria resposta anterior** antes de responder novamente.

## Como funciona:
1. O modelo gera uma resposta inicial
2. Em seguida, critica e reflete sobre os erros ou lacunas dessa resposta
3. Por fim, produz uma resposta melhorada baseada nessa autorreflexão

## Objetivo principal:
Melhorar significativamente a **qualidade, precisão e coerência** das respostas através desse ciclo de feedback automático, permitindo que o modelo "pense melhor" antes de dar a resposta final.

In [5]:
question = """Analise a oportunidade de investimento em Petrobras (PETR4).
Considere:
- Preço atual: R$ 38,50
- P/L: 3,2
- Dividend Yield: 12%
- Contexto: Preços de petróleo estáveis, transição energética global

Forneça recomendação: COMPRA, VENDA ou MANTER."""

domain = "análise de ações brasileiras"

###**Geração da resposta inicial**

In [6]:
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um analista financeiro sênior especializado em {domain}.
    Sua análise deve ser fundamentada, objetiva e baseada em dados.
    Considere sempre fundamentos, valuation, riscos e contexto macroeconômico."""),
    ("human", "{question}")
])

initial_chain = generation_prompt | llm
initial_answer = initial_chain.invoke(
    {
        "question": question,
        "domain": domain
        }
    ).content

###**Geração da avaliação da resposta**

In [7]:
reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gestor de risco rigoroso fazendo "Red Team" da análise abaixo.
    Identifique:
    - Vieses cognitivos (ancoragem, confirmação, otimismo excessivo)
    - Riscos subestimados ou não mencionados
    - Premissas questionáveis ou não testadas
    - Cenários adversos ignorados
    - Pontos fracos na análise

    Seja crítico mas construtivo."""),
    ("human", "Pergunta: {question}\n\nResposta: {answer}\n\nForneça uma crítica construtiva:")
])

reflection_chain = reflection_prompt | llm
critique = reflection_chain.invoke({
    "question": question,
    "answer": initial_answer
}).content

###**Refinamento da resposta com base na avaliação anterior**

In [8]:
improvement_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um Chief Investment Officer (CIO) experiente.
    Melhore a análise original considerando a crítica recebida.

    Sua análise refinada deve incluir:
    - Análise de cenários (otimista/base/pessimista)
    - Riscos claramente identificados e quantificados
    - Recomendação com justificativa robusta
    - Métricas e níveis de preço relevantes"""),
    ("human", "Pergunta: {question}\n\nResposta Original: {answer}\n\nCrítica: {critique}\n\nForneça uma resposta melhorada:")
])

improvement_chain = improvement_prompt | llm
improved_answer = improvement_chain.invoke({
    "question": question,
    "answer": initial_answer,
    "critique": critique
}).content

In [9]:
print({
    "question": question,
    "initial_answer": initial_answer,
    "critique": critique,
    "improved_answer": improved_answer
})

{'question': 'Analise a oportunidade de investimento em Petrobras (PETR4).\nConsidere:\n- Preço atual: R$ 38,50\n- P/L: 3,2\n- Dividend Yield: 12%\n- Contexto: Preços de petróleo estáveis, transição energética global\n\nForneça recomendação: COMPRA, VENDA ou MANTER.', 'initial_answer': '### Análise de Investimento em Petrobras (PETR4)\n\n#### Contexto Atual\nA Petrobras (PETR4) é uma das maiores empresas de petróleo e gás do Brasil, com uma forte influência no mercado energético nacional e internacional. Atualmente, a empresa opera em um ambiente de preços de petróleo relativamente estáveis, mas enfrenta desafios relacionados à transição energética global, que pode impactar a demanda por combustíveis fósseis no longo prazo.\n\n#### Fundamentos\n1. **P/L (Preço sobre Lucro)**: O P/L de 3,2 é extremamente baixo quando comparado ao P/L médio do setor, que normalmente gira em torno de 10 a 15. Um P/L tão baixo pode indicar que a ação está subvalorizada, mas também pode refletir preocupaçõe

##**Refatorando o codigo anterior**

###**Registro de Prompts com LangSmith**

**Prompt Registry** é um conceito de gerenciamento de prompts que funciona como um repositório centralizado para armazenar, versionar e compartilhar templates de prompts.

No contexto do seu código com LangSmith, o prompt registry oferece:

**Principais benefícios:**

- **Versionamento**: Mantém histórico de alterações nos prompts, permitindo rollback se necessário
- **Centralização**: Um único local para gerenciar todos os prompts da organização
- **Reutilização**: Prompts podem ser compartilhados entre diferentes projetos e equipes
- **Colaboração**: Múltiplos desenvolvedores podem trabalhar e iterar sobre os mesmos prompts
- **Rastreabilidade**: Acompanha qual versão do prompt foi usada em cada execução
- **A/B Testing**: Facilita testar diferentes versões de prompts
- **Governança**: Controle de acesso e aprovação de mudanças em prompts

**Como funciona no LangSmith:**

1. Você "registra" (push) um prompt com um identificador único
2. O LangSmith armazena esse prompt na nuvem
3. Qualquer aplicação pode "puxar" (pull) esse prompt usando o identificador
4. Mudanças no prompt são versionadas automaticamente


In [10]:
client = Client()

# prompt = ChatPromptTemplate.from_template("Conte-me uma piada sobre {topic}")
# url = client.push_prompt(
#     prompt_identifier="meu-gerador-de-piadas",
#     object=prompt,
#     tags=[datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
#     )

# print(url)

###**Carregamento do Prompt**

In [11]:
prompt = client.pull_prompt(
    "meu-gerador-de-piadas",
    include_model=True
    )

In [12]:
prompt

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'meu-gerador-de-piadas', 'lc_hub_commit_hash': '2c1e666c6082dead050d0df29158b7706ca5c5f678d67b0c12686e2866fe1a1c'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Conte-me uma piada sobre {topic}'), additional_kwargs={})])

### **Registrando os prompts anteriores**

In [13]:
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um analista financeiro sênior especializado em {domain}.
    Sua análise deve ser fundamentada, objetiva e baseada em dados.
    Considere sempre fundamentos, valuation, riscos e contexto macroeconômico."""),
    ("human", "{question}")
])

reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gestor de risco rigoroso fazendo "Red Team" da análise abaixo.
    Identifique:
    - Vieses cognitivos (ancoragem, confirmação, otimismo excessivo)
    - Riscos subestimados ou não mencionados
    - Premissas questionáveis ou não testadas
    - Cenários adversos ignorados
    - Pontos fracos na análise

    Seja crítico mas construtivo."""),
    ("human", "Pergunta: {question}\n\nResposta: {initial_answer}\n\nForneça uma crítica construtiva:")
])

improvement_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um Chief Investment Officer (CIO) experiente.
    Melhore a análise original considerando a crítica recebida.

    Sua análise refinada deve incluir:
    - Análise de cenários (otimista/base/pessimista)
    - Riscos claramente identificados e quantificados
    - Recomendação com justificativa robusta
    - Métricas e níveis de preço relevantes"""),
    ("human", "Pergunta: {question}\n\nResposta Original: {initial_answer}\n\nCrítica: {critique}\n\nForneça uma resposta melhorada:")
])

## **PromptManager: Gerenciador Centralizado de Prompts**

Esta classe implementa um **padrão de gerenciamento centralizado** para templates de prompts usando a plataforma LangSmith.

### Funcionamento Macro:

**1. Inicialização (`__init__`)**
- Cria uma instância do cliente LangSmith
- Estabelece conexão com o serviço de registro de prompts na nuvem
- Prepara o gerenciador para operações de push/pull

**2. Registro de Prompts (`prompt_registry`)**
- Permite **publicar** templates de prompts no repositório LangSmith
- Cada prompt recebe um identificador único (nome)
- O template é armazenado centralmente e versionado automaticamente
- Funciona como um "git push" para prompts

**3. Recuperação de Prompts (`pull_registered_prompt`)**
- Permite **baixar** templates previamente registrados
- Busca pelo identificador único do prompt
- Retorna o template completo, incluindo metadados do modelo
- Funciona como um "git pull" para prompts

### Objetivo e Benefícios:

Esta classe abstrai a complexidade de gerenciar prompts ao:
- Centralizar o acesso ao LangSmith em uma interface simples
- Padronizar operações de registro e recuperação
- Facilitar versionamento e compartilhamento de prompts entre projetos
- Permitir que equipes colaborem usando os mesmos templates
- Garantir consistência: todos usam a mesma versão do prompt


In [14]:
class PromptManager:
    """Manages prompt templates through LangSmith client operations.

    This class provides a centralized interface for registering and retrieving
    prompt templates using the LangSmith platform.
    """

    def __init__(self) -> None:
        """Initialize the PromptManager with a LangSmith client instance."""
        self._prompt_client = Client()

    def prompt_registry(
        self,
        prompt_name: str,
        prompt: BasePromptTemplate
    ) -> None:
        """Register a prompt template to the LangSmith platform.

        Args:
            prompt_name: Unique identifier for the prompt template.
            prompt: The prompt template object to be registered.
        """
        self._prompt_client.push_prompt(
            prompt_identifier=prompt_name,
            object=prompt
        )

    def pull_registered_prompt(self, prompt_name: str) -> BasePromptTemplate:
        """Retrieve a registered prompt template from LangSmith.

        Args:
            prompt_name: Unique identifier of the prompt template to retrieve.

        Returns:
            The retrieved prompt template object with model information included.
        """
        return self._prompt_client.pull_prompt(
            prompt_identifier=prompt_name,
            include_model=True
        )

In [15]:
prompt_manager = PromptManager()

##**Registrando os Prompt**

In [16]:
# print('Registrando os prompts...')
# prompt_manager.prompt_registry(
#     prompt_name='generation_prompt',
#     prompt=generation_prompt
#     )

# prompt_manager.prompt_registry(
#     prompt_name='reflection_prompt',
#     prompt=reflection_prompt
#     )

# prompt_manager.prompt_registry(
#     prompt_name='improvement_prompt',
#     prompt=improvement_prompt
#     )
# print('Registro de prompts finalizado!')

##**Usando um dos prompts registrados**

In [17]:
prompt_manager.pull_registered_prompt('generation_prompt')

ChatPromptTemplate(input_variables=['domain', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'generation_prompt', 'lc_hub_commit_hash': '9dbf71042b77d041c33a77b148255e727d3e4187d9ee9c5d369f73d4c41eb8ef'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['domain'], input_types={}, partial_variables={}, template='Você é um analista financeiro sênior especializado em {domain}.\n    Sua análise deve ser fundamentada, objetiva e baseada em dados.\n    Considere sempre fundamentos, valuation, riscos e contexto macroeconômico.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

###**Refatorando o codigo anterior**

In [18]:
%%time
generation_prompt = prompt_manager.pull_registered_prompt('generation_prompt')
reflection_prompt = prompt_manager.pull_registered_prompt('reflection_prompt')
improvement_prompt = prompt_manager.pull_registered_prompt('improvement_prompt')

reflexion_chain = (
    RunnablePassthrough.assign(
        initial_answer=generation_prompt | llm | (lambda x: x.content)
    )
    | RunnablePassthrough.assign(
        critique=reflection_prompt | llm | (lambda x: x.content)
    )
    | RunnablePassthrough.assign(
        improved_answer=improvement_prompt | llm | (lambda x: x.content)
    )
    | RunnableLambda(lambda x: {
        "question": x["question"],
        "initial_answer": x["initial_answer"],
        "critique": x["critique"],
        "improved_answer": x["improved_answer"]
    })
)

result = reflexion_chain.invoke({
    "question": question,
    "domain": domain
})

CPU times: user 454 ms, sys: 78.1 ms, total: 532 ms
Wall time: 50.6 s


In [19]:
result

{'question': 'Analise a oportunidade de investimento em Petrobras (PETR4).\nConsidere:\n- Preço atual: R$ 38,50\n- P/L: 3,2\n- Dividend Yield: 12%\n- Contexto: Preços de petróleo estáveis, transição energética global\n\nForneça recomendação: COMPRA, VENDA ou MANTER.',
 'initial_answer': '### Análise da Oportunidade de Investimento em Petrobras (PETR4)\n\n#### 1. Contexto Geral\nA Petrobras (PETR4) é uma das maiores empresas de petróleo e gás do Brasil e uma das principais do mundo. O cenário atual apresenta preços de petróleo estáveis, o que é um fator positivo para a empresa, dado que a volatilidade nos preços do petróleo pode impactar fortemente a receita e os lucros. Além disso, a transição energética global está em curso, o que traz desafios e oportunidades para empresas do setor de energia.\n\n#### 2. Fundamentais da Petrobras\n- **P/L (Preço sobre Lucro)**: 3,2\n  - Este índice indica que a empresa está sendo negociada a um múltiplo muito baixo em relação ao seu lucro, sugerindo 

## **Self Consistency**
Melhora a confiabilidade das respostas de LLMs através de múltiplas gerações e agregação.

### Objetivo da Self-Consistency:

Ao gerar múltiplas respostas independentes:
- Identifica padrões consistentes (pontos que aparecem em várias análises)
- Detecta outliers ou raciocínios questionáveis
- Aumenta confiança em conclusões que emergem repetidamente
- Revela nuances e perspectivas diferentes sobre o mesmo problema

In [20]:
question="""
Um investidor deseja alocar 10% do portfólio em Bitcoin.

Dados:
- Patrimônio: R$ 500.000
- Perfil: Moderado
- Horizonte: 5 anos
- Já possui: 60% renda fixa, 30% ações

Esta alocação é adequada?"""

num_samples=3

##**Criação dos prompts**
- Analise financeira
- Agregação de respostas

In [21]:
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um analista financeiro. Analise a questão passo a passo:

    1. Identifique as variáveis relevantes
    2. Considere os dados disponíveis
    3. Aplique raciocínio lógico e quantitativo
    4. Avalie riscos e oportunidades
    5. Chegue a uma conclusão fundamentada

    Pense em voz alta e mostre seu raciocínio.
    No final, forneça sua resposta final após 'RESPOSTA FINAL:'."""),
    ("human", "{question}")
])

aggregation_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um comitê de investimentos analisando múltiplas opiniões.
    Identifique:
    - Pontos de consenso entre as análises
    - Divergências importantes
    - A conclusão mais robusta e fundamentada
    - Nível de confiança na recomendação

    Forneça a decisão final consolidada."""),
    ("human", "Pergunta: {question}\n\nRespostas geradas:\n{responses}\n\nQual é a resposta mais consistente e por quê?")
])

##**Geração de respostas distintas**


Este código implementa a técnica **Self-Consistency**, que melhora a confiabilidade das respostas de LLMs através de múltiplas gerações e agregação.

### Funcionamento Macro:

**1. Geração de Múltiplas Respostas Diversas**
- Executa um loop gerando `num_samples` respostas diferentes
- Cada iteração usa uma temperatura crescente (0.7, 0.8, 0.9...)
- Temperaturas diferentes = raciocínios diversos para a mesma pergunta

**2. Por que Variar a Temperatura?**
- Temperatura mais baixa (0.7): respostas mais conservadoras e focadas
- Temperatura mais alta (0.9+): respostas mais criativas e exploratórias
- Isso garante diversidade nos caminhos de raciocínio (CoT - Chain of Thought)

**3. Agregação das Respostas**
- Todas as respostas são coletadas em uma lista
- São formatadas e concatenadas com separadores visuais (`---`)
- Cada análise é numerada para fácil referência

In [22]:
responses = []
for i in range(num_samples):
    temp_llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7 + (i * 0.1),
    )
    chain = cot_prompt | temp_llm
    response = chain.invoke({"question": question})
    responses.append(response.content)

formatted_responses = "\n\n---\n\n".join([
    f"Análise {i+1}:\n{resp}" for i, resp in enumerate(responses)
])

## Agregação e Síntese das Respostas (Self-Consistency)

Este código implementa a **fase de agregação** da técnica Self-Consistency, onde múltiplas respostas são sintetizadas em uma resposta final consolidada.

### Funcionamento Macro:

**1. Criação da Chain de Agregação**
- Combina um prompt especializado (`aggregation_prompt`) com o LLM
- Este prompt instrui o modelo a analisar e sintetizar múltiplas respostas
- Usa LCEL (LangChain Expression Language) com o operador `|`

**2. Invocação da Síntese**
- Passa a pergunta original e todas as respostas formatadas para o LLM
- O modelo atua como um "juiz" ou "sintetizador"
- Identifica padrões consistentes, pontos divergentes e gera uma resposta unificada

**3. Output Estruturado**
- Apresenta a pergunta original para contexto
- Mantém todas as respostas individuais para rastreabilidade
- Inclui a resposta final sintetizada (produto da agregação)
- Documenta quantas amostras foram usadas na análise

In [23]:
aggregation_chain = aggregation_prompt | llm

final_answer = aggregation_chain.invoke({
    "question": question,
    "responses": formatted_responses
}).content

print({
    "question": question,
    "individual_responses": responses,
    "final_answer": final_answer,
    "num_samples": num_samples
})

{'question': '\nUm investidor deseja alocar 10% do portfólio em Bitcoin.\n\nDados:\n- Patrimônio: R$ 500.000\n- Perfil: Moderado\n- Horizonte: 5 anos\n- Já possui: 60% renda fixa, 30% ações\n\nEsta alocação é adequada?', 'individual_responses': ['Vamos analisar a situação passo a passo.\n\n### 1. Identifique as variáveis relevantes\n- **Patrimônio Total:** R$ 500.000\n- **Porcentagem a ser alocada em Bitcoin:** 10%\n- **Perfil do investidor:** Moderado\n- **Horizonte de investimento:** 5 anos\n- **Ativos já existentes:**\n  - Renda fixa: 60%\n  - Ações: 30%\n  \n### 2. Considere os dados disponíveis\n- **Valor a ser investido em Bitcoin:** 10% de R$ 500.000 = R$ 50.000\n- **Distribuição atual do portfólio:**\n  - Renda fixa: R$ 300.000 (60%)\n  - Ações: R$ 150.000 (30%)\n  - Bitcoin (proposto): R$ 50.000 (10%)\n\n### 3. Aplique raciocínio lógico e quantitativo\nCom a alocação proposta, o investidor teria:\n- 60% em renda fixa, o que é consistente com um perfil moderado, já que a renda 

## **Reflexion + Self Consistency**

In [24]:

def reflexion(
        question: str,
        domain: str="mercado financeiro"
        ) -> dict:
    generation_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""Você é um analista financeiro sênior especializado em {domain}.
        Sua análise deve ser fundamentada, objetiva e baseada em dados.
        Considere sempre fundamentos, valuation, riscos e contexto macroeconômico."""),
        ("human", "{question}")
    ])

    initial_chain = generation_prompt | llm
    initial_answer = initial_chain.invoke({"question": question}).content

    reflection_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um gestor de risco rigoroso fazendo "Red Team" da análise abaixo.
        Identifique:
        - Vieses cognitivos (ancoragem, confirmação, otimismo excessivo)
        - Riscos subestimados ou não mencionados
        - Premissas questionáveis ou não testadas
        - Cenários adversos ignorados
        - Pontos fracos na análise

        Seja crítico mas construtivo."""),
        ("human", "Pergunta: {question}\n\nResposta: {answer}\n\nForneça uma crítica construtiva:")
    ])

    reflection_chain = reflection_prompt | llm
    critique = reflection_chain.invoke({
        "question": question,
        "answer": initial_answer
    }).content

    improvement_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um Chief Investment Officer (CIO) experiente.
        Melhore a análise original considerando a crítica recebida.

        Sua análise refinada deve incluir:
        - Análise de cenários (otimista/base/pessimista)
        - Riscos claramente identificados e quantificados
        - Recomendação com justificativa robusta
        - Métricas e níveis de preço relevantes"""),
        ("human", "Pergunta: {question}\n\nResposta Original: {answer}\n\nCrítica: {critique}\n\nForneça uma resposta melhorada:")
    ])

    improvement_chain = improvement_prompt | llm
    improved_answer = improvement_chain.invoke({
        "question": question,
        "answer": initial_answer,
        "critique": critique
    }).content

    return {
        "question": question,
        "initial_answer": initial_answer,
        "critique": critique,
        "improved_answer": improved_answer
    }


def self_consistency(
        question: str,
        num_samples: int=5
        ) -> dict:
    cot_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um analista financeiro. Analise a questão passo a passo:

        1. Identifique as variáveis relevantes
        2. Considere os dados disponíveis
        3. Aplique raciocínio lógico e quantitativo
        4. Avalie riscos e oportunidades
        5. Chegue a uma conclusão fundamentada

        Pense em voz alta e mostre seu raciocínio.
        No final, forneça sua resposta final após 'RESPOSTA FINAL:'."""),
        ("human", "{question}")
    ])

    responses = []
    for i in range(num_samples):
        temp_llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7 + (i * 0.1),
            api_key=os.getenv("OPENAI_API_KEY")
        )
        chain = cot_prompt | temp_llm
        response = chain.invoke({"question": question})
        responses.append(response.content)

    formatted_responses = "\n\n---\n\n".join([
        f"Análise {i+1}:\n{resp}" for i, resp in enumerate(responses)
    ])

    aggregation_prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um comitê de investimentos analisando múltiplas opiniões.
        Identifique:
        - Pontos de consenso entre as análises
        - Divergências importantes
        - A conclusão mais robusta e fundamentada
        - Nível de confiança na recomendação

        Forneça a decisão final consolidada."""),
        ("human", "Pergunta: {question}\n\nRespostas geradas:\n{responses}\n\nQual é a resposta mais consistente e por quê?")
    ])

    aggregation_chain = aggregation_prompt | llm
    final_answer = aggregation_chain.invoke({
        "question": question,
        "responses": formatted_responses
    }).content

    return {
        "question": question,
        "individual_responses": responses,
        "final_answer": final_answer,
        "num_samples": num_samples
    }

In [25]:
question="""Valide esta estratégia de day trade no mini-índice (WIN):

- Operar 10h-16h
- Rompimentos de consolidação (30+ min)
- Volume 2x acima da média
- Stop: 50 pontos | Alvo: 150 pontos
- Máximo 3 operações/dia

É viável? Principais riscos? Capital mínimo?""",
domain="day trade",
num_samples=3

sc_result = self_consistency(question, num_samples)
reflexion_result = reflexion(question, domain)

In [26]:
{
    "question": question,
    "self_consistency_samples": sc_result["individual_responses"],
    "self_consistency_answer": sc_result["final_answer"],
    "reflexion_critique": reflexion_result["critique"],
    "final_refined_answer": reflexion_result["improved_answer"]
}

{'question': ('Valide esta estratégia de day trade no mini-índice (WIN):\n\n- Operar 10h-16h\n- Rompimentos de consolidação (30+ min)\n- Volume 2x acima da média\n- Stop: 50 pontos | Alvo: 150 pontos\n- Máximo 3 operações/dia\n\nÉ viável? Principais riscos? Capital mínimo?',),
 'self_consistency_samples': ['Vamos analisar a estratégia de day trade no mini-índice (WIN) passo a passo.\n\n1. **Identifique as variáveis relevantes**:\n   - **Horário de operação**: 10h-16h\n   - **Critério de entrada**: Rompimentos de consolidação com duração de 30 minutos ou mais\n   - **Volume**: Deve ser 2x acima da média\n   - **Gestão de riscos**: Stop de 50 pontos e alvo de 150 pontos\n   - **Limite de operações**: Máximo de 3 operações por dia\n\n2. **Considere os dados disponíveis**:\n   - O mini-índice (WIN) é conhecido por sua volatilidade, o que pode ser favorável para estratégias de day trade.\n   - O horário de operação escolhido é compreensível, pois geralmente há maior liquidez e volatilidade 